In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
cd /content/gdrive/My\ Drive/Data\ Sets

Using TensorFlow backend.


In [0]:
train = pd.read_csv('train_01/train_new.csv')

In [0]:
train_image = []
for i in tqdm(range(train.shape[0])):
    if i%3 != 0: continue
    img = image.load_img('train_01/'+train['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img) / 255
    train_image.append(img)
train_image = np.array(train_image)

100%|██████████| 33801/33801 [1:25:59<00:00,  6.55it/s]


In [0]:
#np.save('train_image', train_image, fix_imports='false')

In [0]:
y_ = train['class']
y = []
for i in range(len(y_)):
    if i%3 == 0: y.append(y_[i])
X_train, X_test, y_train, y_test = train_test_split(train_image, y, random_state=100, stratify = y, test_size=0.2)
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [0]:
init_model = VGG19(weights='imagenet', include_top=False)
X_train = init_model.predict(X_train)
X_test = init_model.predict(X_test)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]*X_test.shape[3])
mx_train = X_train.max()
X_train /= mx_train
X_test /= mx_train
print(X_train.shape, X_test.shape)





80142336/80134624 [==============================] - 2s 0us/step






(9013, 25088) (2254, 25088)


In [0]:
model = keras.models.Sequential([
    Dense(512, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(101, activation='softmax'),
])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weights2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=256)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 9013 samples, validate on 2254 samples
Epoch 1/150
9013/9013 [==============================] - 5s 516us/step - loss: 4.5482 - acc: 0.0207 - val_loss: 4.3749 - val_acc: 0.0577
Epoch 2/150
9013/9013 [==============================] - 4s 417us/step - loss: 4.2214 - acc: 0.0773 - val_loss: 3.8325 - val_acc: 0.1735
Epoch 3/150
9013/9013 [==============================] - 3s 379us/step - loss: 3.7318 - acc: 0.1579 - val_loss: 3.1838 - val_acc: 0.2995
Epoch 4/150
9013/9013 [==============================] - 3s 382us/step - loss: 3.2153 - acc: 0.2390 - val_loss: 2.6256 - val_acc: 0.3869
Epoch 5/150
9013/9013 [==============================] - 4s 399us/step - loss: 2.7905 - acc: 0.3189 - val_loss: 2.2131 - val_acc: 0.4876
Epoch 6/150
9013/9013 [==============================] - 4s 391us/step - loss: 2.4239 - acc: 0.3901 - val_loss: 1.9502 - val_acc: 0.5563
Epoch 7/150
9013/9013 [=========